In [1]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

import argparse
args = argparse.Namespace()
args.raw_train = pd.read_csv('../data/train.csv.zip')
args.raw_test = pd.read_csv('../data/test.csv.zip')
args.predictions_folder = Path('../predictions')
args.n_splits = 5
args.n_jobs = 3
args.seed = 101
args.val_fraction=0.1
args.epochs = 50

args.train = args.raw_train.iloc[:, 1:].copy()
args.labels = args.raw_train['label'].copy()
args.test = args.raw_test.copy()


args.predictions_folder.mkdir(parents=True, exist_ok=True) 

In [5]:
X = args.train.to_numpy().reshape(args.train.shape[0], 28, 28, 1)
y = pd.get_dummies(args.labels, prefix='label').to_numpy()
y_sparse = args.labels.to_numpy()
x = args.test.to_numpy().reshape(args.test.shape[0], 28, 28, 1)

In [3]:
t = tf.constant([[1, 2], [3, 4], [5, 6]])
tf.square(t)
keras.backend.transpose(t)
t + tf.cast(tf.constant(1.), tf.int32)
v = tf.Variable([[1, 2], [3, 4], [5, 6]])
v[0,0].assign(100)
tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
tf.constant(['019A'])
# tf.strings.unicode_decode(tf.constant(['019A']), input_encoding='UTF-8')

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'019A'], dtype=object)>

### Custom training loops

In [8]:
len(X) // 32
32 * 1312

41984

In [17]:
def shuffle(X, y):
    x = np.array([0, 100, 200, 300, 400])
    a = np.random.permutation(5)
    print(a)
    print(x[a])

for i in range(5):
    shuffle(None, None)

[3 4 0 1 2]
[300 400   0 100 200]
[2 4 1 3 0]
[200 400 100 300   0]
[3 1 0 2 4]
[300 100   0 200 400]
[1 0 4 3 2]
[100   0 400 300 200]
[3 0 4 2 1]
[300   0 400 200 100]


In [4]:
def build_simple_model():
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(28, 28)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'))
    model.add(keras.layers.Dense(10, activation='softmax')) # !
    return model
#     model.compile(loss='sparse_categorical_crossentropy', # !
#              optimizer=keras.optimizers.Nadam(learning_rate=lr),
#              metrics=[keras.metrics.sparse_categorical_accuracy]) # !

def custom_training_loop():
    def shuffle(X, y):
        assert len(X) == len(y)
        idx = np.random.permutation(len(X))
        return X[idx], y[idx]
    
    n_epochs = 5
    batch_size = 32
    optimizer=keras.optimizers.Nadam(learning_rate=0.001)
    loss_fn = keras.losses.sparse_categorical_crossentropy # try class
    metrics=[keras.metrics.SparseCategoricalAccuracy] # try func
    
    for epoch in range(1, n_epochs + 1):
        log.info(f'Epoch {epoch}...')
        n_steps = len(X) // 32 + 1
        X_shuffled, y_shuffled = shuffle(X, y)
        for step in range(1, n_steps + 1):
            ceiling = step * batch_size if ceiling < len(X_shuffled) else len(X_shuffled)
            floor = (step - 1) * batch_size
    
    